<a href="https://colab.research.google.com/github/bellomusodiq/machine_learning/blob/master/text_generation_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
from keras.preprocessing import sequence
from tensorflow import keras
import tensorflow as tf
import os 
import numpy as np

Using TensorFlow backend.


In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [0]:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

In [0]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('lenght of text: {} characters'.format(len(text)))

lenght of text: 1115394 characters


In [0]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
vocab = sorted(set(text))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [0]:
print("Text:", text[:20])
print("Encoded:", text_to_int(text[:20]))

Text: First Citizen:
Befor
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56]


In [0]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

int_to_text([35,54,6,22])

'Wp,J'

In [0]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
for data in char_dataset.take(13):
  print(data)

tf.Tensor(18, shape=(), dtype=int64)
tf.Tensor(47, shape=(), dtype=int64)
tf.Tensor(56, shape=(), dtype=int64)
tf.Tensor(57, shape=(), dtype=int64)
tf.Tensor(58, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(15, shape=(), dtype=int64)
tf.Tensor(47, shape=(), dtype=int64)
tf.Tensor(58, shape=(), dtype=int64)
tf.Tensor(47, shape=(), dtype=int64)
tf.Tensor(64, shape=(), dtype=int64)
tf.Tensor(43, shape=(), dtype=int64)
tf.Tensor(52, shape=(), dtype=int64)


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
  input_text = chunk[:-1] # hell
  target_text = chunk[1:] # ello
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for x, y in dataset.take(2):
  print('Input')
  print(int_to_text(x.numpy()))
  print('Target')
  print(int_to_text(y.numpy()))

Input
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
Target
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
Input
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 
Target
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [0]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Creating Loss
for input, target in data.take(1):
  example_batch_predictions = model(input)
  print(example_batch_predictions.shape)

(64, 100, 65)


In [0]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 2.9944372e-03 -4.5068813e-03 -3.1000827e-03 ... -3.1774025e-04
   -1.6128606e-03  1.1124160e-03]
  [ 1.1216357e-03 -5.1360452e-03 -3.8138372e-03 ...  1.2319759e-03
   -4.6296790e-03  1.8391861e-03]
  [ 1.6608776e-03 -6.5122126e-03 -2.0206091e-04 ... -1.5949495e-03
   -8.4232176e-03  1.9416037e-03]
  ...
  [ 1.0853268e-02 -1.2408068e-02  6.3565923e-03 ... -5.0725862e-03
   -9.2509948e-04 -7.0947437e-03]
  [ 4.2753620e-03 -1.0337851e-02  1.0707083e-03 ... -9.3835695e-03
    3.8445895e-03 -5.9293816e-03]
  [ 5.5078398e-03 -1.1233082e-02  3.1831919e-03 ... -6.0729212e-03
   -3.2890388e-03 -4.4029793e-03]]

 [[ 3.4486677e-03 -1.6382255e-03 -8.9859311e-04 ...  1.8297890e-03
    3.1552776e-03 -1.8000109e-03]
  [-9.7283430e-04 -1.7215237e-03 -3.5312944e-03 ... -4.4550165e-03
    7.8550149e-03 -1.7217596e-03]
  [-1.6082181e-03 -3.8894424e-03 -1.8241321e-03 ... -4.2194356e-03
    1.2896698e-02 -1.9532838e-03]
  ...
  [ 4.6825632e-03 -4.3716710e-03  5.6542465e-03 ... -8.3767613e

In [0]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[ 0.00299444 -0.00450688 -0.00310008 ... -0.00031774 -0.00161286
   0.00111242]
 [ 0.00112164 -0.00513605 -0.00381384 ...  0.00123198 -0.00462968
   0.00183919]
 [ 0.00166088 -0.00651221 -0.00020206 ... -0.00159495 -0.00842322
   0.0019416 ]
 ...
 [ 0.01085327 -0.01240807  0.00635659 ... -0.00507259 -0.0009251
  -0.00709474]
 [ 0.00427536 -0.01033785  0.00107071 ... -0.00938357  0.00384459
  -0.00592938]
 [ 0.00550784 -0.01123308  0.00318319 ... -0.00607292 -0.00328904
  -0.00440298]], shape=(100, 65), dtype=float32)


In [0]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[ 2.9944372e-03 -4.5068813e-03 -3.1000827e-03 -7.7587593e-04
 -3.4814174e-03 -5.3836085e-04  7.1866480e-03  5.9816323e-04
 -2.2754211e-03  4.0485733e-03 -2.3087640e-03  2.5940430e-03
  3.1151713e-03 -3.8202509e-04  1.6324642e-03 -1.4141508e-03
 -1.5222550e-03  1.3013456e-03  3.0231890e-03  6.2615396e-03
 -3.4406469e-03 -3.3368871e-03  2.2406637e-04  5.6786863e-03
  9.5779949e-05 -3.6917701e-03 -1.3519300e-03  9.3230035e-04
 -1.9535606e-03 -2.6999391e-04  1.2188258e-03 -8.7019498e-04
 -3.3125200e-04  2.8198338e-03 -1.3978061e-03 -5.1671248e-03
  1.2175059e-03 -3.3976950e-03 -3.7551245e-03  1.2030629e-03
 -5.8415718e-03  2.3232882e-03  3.6471770e-03  4.1435822e-03
  2.2625877e-03  9.8045953e-03 -1.5206988e-03 -1.6148288e-03
 -5.3044409e-03 -1.1012729e-03 -7.6781888e-04 -1.7113908e-04
  3.6996393e-03  2.4976657e-04 -1.2880901e-03  2.5610146e-03
  4.9839900e-03  3.6135672e-03  2.1634914e-03  4.5661489e-03
  1.1059704e-03 -3.0801084e-04 -3.1774025e-04 -1.6128606e-03
  1.112416

In [0]:
# If 
sampled_indices = tf.random.categorical(pred, num_samples=1)

sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

'IKzeNLCo$QHaWfxxQ:;t\n?SjdMYm go$?pRpppO.nAtXI;Us3ysQy3XOGIys?gXSBTNPcw,obG.f .MySQHnHw3! BgeKHr.-xFl'

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [0]:
model.fit(data, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 9s 53ms/step - loss: 2.5756
Epoch 2/10
172/172 [==============================] - 9s 53ms/step - loss: 1.8796
Epoch 3/10
172/172 [==============================] - 9s 53ms/step - loss: 1.6307
Epoch 4/10
172/172 [==============================] - 9s 53ms/step - loss: 1.4990
Epoch 5/10
172/172 [==============================] - 9s 53ms/step - loss: 1.4203
Epoch 6/10
172/172 [==============================] - 9s 53ms/step - loss: 1.3646
Epoch 7/10
172/172 [==============================] - 9s 53ms/step - loss: 1.3204
Epoch 8/10
172/172 [==============================] - 9s 53ms/step - loss: 1.2820
Epoch 9/10
172/172 [==============================] - 9s 53ms/step - loss: 1.2445
Epoch 10/10
172/172 [==============================] - 9s 53ms/step - loss: 1.2091


In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, 1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_2 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = .65

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u" "))

 here.

GREMIO:
Who comes it that falls the grace of all.

HERMIONE:
I take him and so.

JULIET:
He's standance is set down the city.

PETRUCHIO:
Come, you are as his force and strange news as in a braverent
As I can do it, lords: the other of my husband
Which thereof will speak a man; and here is mine and blood.

JULIET:
It is a good heart him in a husband's land,
That fear the sight of comfort but the castle.

DUKE VINCENTIO:
Sir, I come to him; there is my never be not might be so,
That is't unseen'd with him: he is a sin in in an hand,
And something that we had merely in death.

GREMIO:
Didst thou learn in such a constant death?

Second Musician:
Farewell, I think, I speak, you have no less
Than in his report in the fixe of mine,
Who with deceived his design for me.

LADY ANNE:
And I to see thee might to be constant to come on.

GLOUCESTER:
I do repent thee or the else that they have no joy
That would have given them to the father's life,
He perished in a pilth, die.

ROMEO:
What, 